# Create and add data to S3 Bucket 

In [ ]:
           location string,
                title string,
                channelTitle string,
                categoryid int,
                tags string,
                view_count string,
                likes int,
                dislikes int,	
                comment_count int,
                comments_disabled boolean,
                ratings_disabled boolean,
                description_length int,
                timepublished string,
                trending_month string,
                trending_year string)
                

In [ ]:
with open('CA_category_id.json') as file:
    data = DataFrame([id["id"] for id in json.loads(file.read())["items"]])
print(data.head(3))

    0
0   1
1   2
2  10


In [69]:
with open('CA_category_id.json') as file:
    data2 = DataFrame([snippet["snippet"] for snippet in json.loads(file.read())["items"]])
    data3 = data2['title']
print(data3.head(3))

0    Film & Animation
1    Autos & Vehicles
2               Music
Name: title, dtype: object


In [70]:
category_final = pd.concat([data, data3], axis=1, join='inner')
category_final.columns =['id_number', 'catagory_name']
category_final.head(3)

,id_number,catagory_name
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music


# Reformat 'tags' 'title' and 'channelTitle' features
##### the Tags feature also presented a problem with athena
##### One solution i found is to just strip all of the character from the column so that it is an uninterupted string for each instance

In [136]:
def strip(x):
    #removes all the formatting so I can search against a "clean" list
    x = x.str.replace(' ','', regex=True).str.replace('.','', regex=True).str.replace(',','', regex=True).str.replace('|','', regex=True).str.replace("'",'', regex=True).str.replace('-',"", regex=True).str.replace("(",'', regex=True).str.replace(')',"", regex=True).str.replace('/',"", regex=True).str.replace('â„¢',"", regex=True).str.replace('â‚¬',"", regex=True).str.replace('Ã¢',"", regex=True).str.replace('&',"", regex=True).str.lower().str.replace('-',"", regex=True).str.replace('&',"", regex=True)
    return x

In [137]:
finalcsv['tags'] = strip(finalcsv['tags'])
finalcsv['title'] = strip(finalcsv['title'])
finalcsv['channelTitle'] = strip(finalcsv['channelTitle'])

In [ ]:
### Move the CategoryId column to the end
#finalcsv.insert(len(finalcsv.columns)-1, 'categoryId', finalcsv.pop('categoryId'))

In [ ]:
def strip(x):
    #removes all the formatting so I can search against a "clean" list
    x = x.str.replace(' ','', regex=True).str.replace('.','', regex=True).str.replace(',','', regex=True).str.replace("'",'', regex=True).str.replace('-',"", regex=True).str.replace("(",'', regex=True).str.replace(')',"", regex=True).str.replace('/',"", regex=True).str.replace('â„¢',"", regex=True).str.replace('â‚¬',"", regex=True).str.replace('Ã¢',"", regex=True).str.replace('&',"", regex=True).str.lower().str.replace('-',"", regex=True).str.replace('&',"", regex=True)
    return x
ca['title'] = strip(ca['title'])
gb['title'] = strip(gb['title'])
us['title'] = strip(us['title'])
ca['channelTitle'] = strip(ca['channelTitle'])
gb['channelTitle'] = strip(gb['channelTitle'])
us['channelTitle'] = strip(us['channelTitle'])

### Create Bucket

In [38]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

In [47]:
print(region)

us-east-1


### Verify Bucket Creation

In [39]:
%%bash

#aws s3 ls s3://${bucket}/

### Verify Connection to bucket

In [40]:
print("Default bucket: {}".format(bucket))

Default bucket: sagemaker-us-east-1-418795740224


### *Manually Add data into S3 bucket on AWS site*

### Make sure the data is in the Bucket

In [41]:
%%bash

#aws s3 ls s3://sagemaker-us-east-1-418795740224/data/
aws s3 ls s3://${bucket}/data/

2023-03-09 17:18:57 sagemaker-us-east-1-418795740224


### Store Data path as a String Variable

In [42]:
datapath = "s3://sagemaker-us-east-1-418795740224/data/"
%store datapath

Stored 'datapath' (str)


# Data Transformation and CSV aggregation

### Load CSV files in s3 into pandas dataframes

In [43]:
ca = pd.read_csv("s3://sagemaker-us-east-1-418795740224/data/CA_youtube_trending_data.csv")
gb = pd.read_csv("s3://sagemaker-us-east-1-418795740224/data/GB_youtube_trending_data.csv")
us = pd.read_csv("s3://sagemaker-us-east-1-418795740224/data/US_youtube_trending_data.csv")

us.insert(0, 'Location',  'US')
gb.insert(0, 'Location',  'GB')
ca.insert(0, 'Location',  'CA')

#response = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key="files/books.csv")
#status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")
#if status == 200:
   # print(f"Successful S3 get_object response. Status - {status}")
   # books_df = pd.read_csv(response.get("Body"))
   # print(books_df)
#else:
   # print(f"Unsuccessful S3 get_object response. Status - {status}")

In [44]:
us.insert(0, 'Location',  'US')
gb.insert(0, 'Location',  'GB')
ca.insert(0, 'Location',  'CA')

# Create Athena Databae

### Dependencies

In [13]:
!pip install pyathena
from pyathena import connect
import pandas as pd

  Using cached pyathena-2.23.0-py3-none-any.whl (67 kB)


### Define database name

In [41]:
dbname = "Viewboost"

### Establish Athena's tempoary Working Directory

In [37]:
wd = datapath + "athena/staging"

### Connect to Athena

In [38]:
conn = connect(region_name=region, s3_staging_dir=wd)

### Create SQL Statement for database creation

In [42]:
sql = "CREATE DATABASE IF NOT EXISTS {}".format(dbname)

### Use pandas to run the statment

In [45]:
#pd.read_sql(sql, conn)

### Verify Database creation

In [47]:
sql2 = "SHOW DATABASES"
pd.read_sql(sql2, conn)

,database_name
0,default
1,dsoaws
2,youtubedb


### Store the Database

In [46]:
%store Viewboost

UsageError: Unknown variable 'Viewboost'
